In [13]:
# 表格：wxg_weixin_analytics_intern::tmp_dws_app_weixin_pridict_feature_fanslevel_uin_distribute_with_strategy_20230324_intern

In [14]:
# 从数据库中读取数据包
from pyspark.sql import SparkSession
import os
from pytoolkit import TDWSQLProvider
import pandas

In [15]:
# 从数据库中读取数据
os.environ['GROUP_ID'] = 'g_wxg_wxt_product_analytics_intern'
os.environ['GAIA_ID'] = '5729'
session = SparkSession.builder.config('spark.driver.maxResultSize','14g').config('spark.driver.memory', '14g').config('spark.executor.cores', 4).config('spark.executor.memory', '14g').config('spark.yarn.archive','hdfs://qy-ieg-4-v2/data/SPARK/cachefile/3_12_2.4.6_hadoop3/spark-jars_old_notauth.zip').getOrCreate()

In [16]:
# 数据量太大，只能不同的SQL分开读取
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='wxg_weixin_analytics_intern')
tbl=provider.table('tmp_dws_app_weixin_pridict_feature_fanslevel_uin_distribute_with_strategy_20230324_intern')
tbl.createOrReplaceTempView('tbl')
df1=session.sql('select * from tbl limit 100').toPandas()

In [17]:
df1.head(5)

,session_time,stay_time_s,rate,feedcnt_f1c1,feedcnt_f2c1,feedcnt_f3c1,feedcnt_f4c1,feedcnt_f5c1,feedcnt_f6c1,feedcnt_f1c2,...,feedcnt_f5c3,feedcnt_f6c3,feedcnt_f1c4,feedcnt_f2c4,feedcnt_f3c4,feedcnt_f4c4,feedcnt_f5c4,feedcnt_f6c4,groupid,exptid
0,-4573.546973,129207.078550,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
1,2212.741011,27221.674663,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
2,2212.741011,1.663547,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
3,2212.741011,1.663547,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
4,2576.964084,9249.296302,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162


In [18]:
#数据分析包
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt

# sklearn 计算mse,r2,mape(y_true,y_predict)
from sklearn.metrics import r2_score

# linear regression
# from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

# sklearn--lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # standardize the predictors
from sklearn.linear_model import Lasso

# EDA

In [19]:
# 查看3个实验，每个实验的各个实验组的id
group_df=session.sql('select groupid,exptid from tbl').toPandas()

# 查看各组数量
print("不同实验：",group_df["exptid"].unique())
print("每个实验对应的group:",group_df[["exptid","groupid"]].value_counts().groupby("exptid").count())

# 查看每个实验对应的实验组
print("粉丝砍量实验：",group_df[group_df["exptid"]==8540098]["groupid"].unique())
print("垂类砍量实验1：",group_df[group_df["exptid"]==8998768]["groupid"].unique())
print("垂类砍量实验2：",group_df[group_df["exptid"]==9457529]["groupid"].unique())
print("拉新渠道实验：",group_df[group_df["exptid"]==9916162]["groupid"].unique())

不同实验： [9916162 9457529 8998768 8540098]
每个实验对应的group: exptid
8540098    20
8998768    17
9457529    16
9916162     5
dtype: int64
粉丝砍量实验： [8540118 8540112 8540121 8540123 8540122 8540127 8540119 8540114 8540125
 8540120 8540115 8540126 8540129 8540117 8540130 8540113 8540116 8540124
 8540128 8540131]
垂类砍量实验1： [9064441 9064434 9064432 9064440 9064439 9064437 9064433 9064436 9064444
 9064438 9064447 9064442 9064445 9064435 9064443 9064446 9064431]
垂类砍量实验2： [9457548 9457556 9457549 9457546 9457553 9457545 9457557 9457555 9457558
 9457554 9457559 9457550 9457544 9457552 9457547 9457551]
拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]


In [20]:
# 查看数据量
print("训练集数据量（垂泪+粉丝等级）：",len(group_df[group_df["exptid"]==8540098])+len(group_df[group_df["exptid"]==8998768])+ len(group_df[group_df["exptid"]==9457529]))
print("验证集数据量：",len(group_df[group_df["exptid"]==9916162]))

group_id_df1=pd.DataFrame(group_df[group_df["exptid"]==8540098]["groupid"].value_counts())
# print("粉丝砍量实验：",group_id_df1)
group_id_df2=pd.DataFrame(group_df[group_df["exptid"]==8998768]["groupid"].value_counts())
# print("垂类砍量实验1：",group_id_df2)
group_id_df3=pd.DataFrame(group_df[group_df["exptid"]==9457529]["groupid"].value_counts())
# print("垂类砍量实验2：",group_id_df3)
group_id_df4=pd.DataFrame(group_df[group_df["exptid"]==9916162]["groupid"].value_counts())
# print("拉新渠道实验：",group_id_df4)

# 将各组的group_id 和对应的数据量存储为df
group_id_df=pd.concat([group_id_df1, group_id_df2])
group_id_df=pd.concat([group_id_df, group_id_df3])
group_id_df=pd.concat([group_id_df, group_id_df4])
group_id_df= group_id_df.reset_index().rename(columns={"index":"group_id","groupid":"number"})

# 确定down sample的数量
group_id_df["downsample_number"]=(group_id_df["number"]/(2.5)).apply(lambda x: int(x) if not pd.isnull(x) else x)
print("原来数据总量：",group_id_df[["number"]].sum())
print("downsample后的数据总量：",group_id_df[["downsample_number"]].sum())
group_id_df.head(5)

训练集数据量（垂泪+粉丝等级）： 9790989
验证集数据量： 1114053
原来数据总量： number    10905042
dtype: int64
downsample后的数据总量： downsample_number    4361995
dtype: int64


,group_id,number,downsample_number
0,8540126,269482,107792
1,8540114,268333,107333
2,8540123,268248,107299
3,8540129,268084,107233
4,8540115,268046,107218


# Down sample

###  1. 按照X分层抽样

### 1.1 直接按照X分层抽样

In [21]:
# down sample 抽样
import random
# 创建一个空的 DataFrame，用于存储抽样后的数据
sampled_df = pd.DataFrame()

for i in range(0,len(group_id_df)):
    temp_groupid= group_id_df.loc[i,"group_id"]
    print(temp_groupid)
    # print(temp_groupid)
    temp_downsample_number= group_id_df.loc[i,"downsample_number"]
    # print(temp_downsample_number)
    temp_data= session.sql(f'select * from tbl where groupid=={temp_groupid}').toPandas()
    # 将列 'stay_time_s' 等分为 4 个分组
    q=3
    bin_edges = np.percentile(temp_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("原始分组",bin_edges)
    if len(np.unique(bin_edges, return_counts=True)[0]) < q+1:
        print('Warning: bin edges are not unique')
        q=q-1
    else:
        print('Bin edges are unique')
        q=q
    bin_edges = np.percentile(temp_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("最后分组",bin_edges)
    bins = pd.qcut(temp_data['stay_time_s'], q=q, labels=False)
    n=round(temp_downsample_number/q)
    temp_sampled_data = pd.concat([group.sample(n,random_state=100) for _, group in temp_data.groupby(bins)])   # random_state=100
    sampled_df = pd.concat([sampled_df, temp_sampled_data])
# 输出抽样后的样本数量
sampled_df=sampled_df.reset_index()
print(f'Total: {len(sampled_df)}')

8540126
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 8.78178679e+06]
最后分组 [1.66354683e+00 3.90126290e+01 8.78178679e+06]
8540114
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 7.21840649e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.21840649e+06]
8540123
原始分组 [1.66354683e+00 1.66354683e+00 1.30141161e+03 1.07778114e+07]
最后分组 [1.66354683e+00 3.90126290e+01 1.07778114e+07]
8540129
原始分组 [1.66354683e+00 1.66354683e+00 1.33129087e+03 9.18189256e+06]
最后分组 [1.66354683e+00 3.90126290e+01 9.18189256e+06]
8540115
原始分组 [1.66354683e+00 1.66354683e+00 1.30141161e+03 7.79863196e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.79863196e+06]
8540120
原始分组 [1.66354683e+00 1.66354683e+00 1.34623051e+03 7.72726533e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.72726533e+06]
8540117
原始分组 [1.66354683e+00 1.66354683e+00 1.35370032e+03 7.51535411e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.51535411e+06]
8540121
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 6.64872094e+06]
最后分组 [1.66354683e+00 3.90126290e+01 

### 1.2 最小值太多，最小值单独抽样。其余组分层抽样

In [ ]:
# down sample 抽样--把最小值按照比率进行抽样，其余的组分为4组
import random
# 创建一个空的 DataFrame，用于存储抽样后的数据
sampled_df = pd.DataFrame()

for i in range(0,len(group_id_df)):
    temp_groupid= group_id_df.loc[i,"group_id"]
    print(temp_groupid)
    # print(temp_groupid)
    temp_downsample_number= group_id_df.loc[i,"downsample_number"]
    # print(temp_downsample_number)
    temp_data= session.sql(f'select * from tbl where groupid=={temp_groupid}').toPandas()
    ###################################
    # 最小值的按比例sample
    temp_data_min= min(temp_data["stay_time_s"])
    temp_data_min_data =temp_data[temp_data["stay_time_s"]==temp_data_min]
    ### 最小的min的数量
    min_downsample_number= round((len(temp_data_min_data)/len(temp_data))*temp_downsample_number)
    temp_min_sample= temp_data_min_data.sample(n=min_downsample_number,random_state=100)  # random_state=100
    ####################################
    # 其余的data按比例sample
    temp_data_rest_data=temp_data[temp_data["stay_time_s"]!=temp_data_min]
    rest_downsample_number=temp_downsample_number-min_downsample_number
    # 将剩余的data按列 'stay_time_s' 等分为 q+1个分组
    q=2
    bin_edges = np.percentile(temp_data_rest_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("原始分组",bin_edges)
    if len(np.unique(bin_edges, return_counts=True)[0]) < q+1:
        print('Warning: bin edges are not unique')
        q=q-1
    else:
        print('Bin edges are unique')
        q=q
    bin_edges = np.percentile(temp_data_rest_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("最后分组",bin_edges)
    bins = pd.qcut(temp_data_rest_data['stay_time_s'], q=q, labels=False)
    n=round(rest_downsample_number/q)
    temp_rest_sample= pd.concat([group.sample(n,random_state=100) for _, group in temp_data.groupby(bins)])   # random_state=100
    #####################################
    # 合并data
    temp_sampled_data = pd.concat([temp_min_sample, temp_rest_sample])
    ######################################
    # 循环合并到sampled_df
    sampled_df = pd.concat([sampled_df, temp_sampled_data])
# 输出抽样后的样本数量
sampled_df=sampled_df.reset_index()
print(f'Total: {len(sampled_df)}')

8540126
原始分组 [9.13336328e+00 4.16982112e+03 8.78178679e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.16982112e+03 8.78178679e+06]
8540114
原始分组 [9.13336328e+00 4.22210984e+03 7.21840649e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.22210984e+03 7.21840649e+06]
8540123
原始分组 [9.13336328e+00 4.17729094e+03 1.07778114e+07]
Bin edges are unique
最后分组 [9.13336328e+00 4.17729094e+03 1.07778114e+07]
8540129
原始分组 [9.13336328e+00 4.23704947e+03 9.18189256e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.23704947e+03 9.18189256e+06]
8540115
原始分组 [9.13336328e+00 4.12500222e+03 7.79863196e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.12500222e+03 7.79863196e+06]
8540120
原始分组 [9.13336328e+00 4.18476075e+03 7.72726533e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.18476075e+03 7.72726533e+06]
8540117
原始分组 [9.13336328e+00 4.17729094e+03 7.51535411e+06]
Bin edges are unique
最后分组 [9.13336328e+00 4.17729094e+03 7.51535411e+06]
8540121
原始分组 [9.13336328e+00 4.15488149e+03 6.64872094e+06]
Bin edges

In [22]:
# 脱敏数据检查

# 拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]
# 拉新渠道：对照组：10047359，10047360； 实验组：group1:10047361; group2:10047362;group3:10047363
group_mean=[]
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))
group_std=[]
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))

# 检查脱敏后的实验组的mean 和 std
check=pd.DataFrame(dict(
    group_mean=group_mean,
    group_std=group_std
))
check

,group_mean,group_std
0,2813.285133,5998.970870
1,2846.624461,5650.058468
2,2848.846172,6027.581842
3,2811.929574,6004.356922
4,2860.525034,5976.228614


In [11]:
sampled_df.to_csv("sampled_df_x.csv")

### 2. 按照Y分层抽样

In [17]:
# down sample 抽样
import random
# 创建一个空的 DataFrame，用于存储抽样后的数据
sampled_df = pd.DataFrame()

for i in range(0,len(group_id_df)):
    temp_groupid= group_id_df.loc[i,"group_id"]
    print(temp_groupid)
    # print(temp_groupid)
    temp_downsample_number= group_id_df.loc[i,"downsample_number"]
    # print(temp_downsample_number)
    temp_data= session.sql(f'select * from tbl where groupid=={temp_groupid}').toPandas()
    # 将列 'stay_time_s' 等分为 4 个分组
    q=2
    bin_edges = np.percentile(temp_data['session_time'], np.linspace(0, 100, q+1))
    print("原始分组",bin_edges)
    if len(np.unique(bin_edges, return_counts=True)[0]) < q+1:
        print('Warning: bin edges are not unique')
        q=q-1
    else:
        print('Bin edges are unique')
        q=q
    bin_edges = np.percentile(temp_data['session_time'], np.linspace(0, 100, q+1))
    print("最后分组",bin_edges)
    bins = pd.qcut(temp_data['session_time'], q=q, labels=False)
    n=round(temp_downsample_number/q)
    temp_sampled_data = pd.concat([group.sample(n) for _, group in temp_data.groupby(bins)])
    sampled_df = pd.concat([sampled_df, temp_sampled_data])
# 输出抽样后的样本数量
sampled_df=sampled_df.reset_index()
print(f'Total: {len(sampled_df)}')

8540126
原始分组 [-117000.64126014    1824.76096376  116718.72992456]
Bin edges are unique
最后分组 [-117000.64126014    1824.76096376  116718.72992456]
8540114
原始分组 [-111788.43409827    1824.76096376  126101.24371096]
Bin edges are unique
最后分组 [-111788.43409827    1824.76096376  126101.24371096]
8540123
原始分组 [-107807.17649355    1824.76096376  122676.61306333]
Bin edges are unique
最后分组 [-107807.17649355    1824.76096376  122676.61306333]
8540129
原始分组 [-119647.45840274    1824.76096376  156695.58567539]
Bin edges are unique
最后分组 [-119647.45840274    1824.76096376  156695.58567539]
8540115
原始分组 [-100365.19581801    1824.76096376  125184.58663977]
Bin edges are unique
最后分组 [-100365.19581801    1824.76096376  125184.58663977]
8540120
原始分组 [-113268.28061124    1824.76096376  122130.62765746]
Bin edges are unique
最后分组 [-113268.28061124    1824.76096376  122130.62765746]
8540117
原始分组 [-98420.18506694   1824.76096376 151758.59903456]
Bin edges are unique
最后分组 [-98420.18506694   1824.76096376 151758.5

In [18]:
# 脱敏数据检查

# 拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]
# 拉新渠道：对照组：10047359，10047360； 实验组：group1:10047361; group2:10047362;group3:10047363
group_mean=[]
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))
group_std=[]
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))

# 检查脱敏后的实验组的mean 和 std
check=pd.DataFrame(dict(
    group_mean=group_mean,
    group_std=group_std
))
check

,group_mean,group_std
0,3494.682295,6741.239520
1,3518.708245,6619.532376
2,3544.611985,6858.928053
3,3543.426018,6886.244942
4,3532.503462,6927.533357


In [ ]:
sampled_df.to_csv("sampled_df_Y.csv")